In [1]:
#Import required packages.

import urllib  
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [2]:
#Setting up parser preliminiaries.

opener=urllib.request.build_opener()
opener.addheaders=[('User-Agent', 'Google Chrome/57.0.2987.133')]

url=('http://www.trademe.co.nz')
ourURL=opener.open(url).read()

In [3]:
def urlmaker(page):
    return('http://www.trademe.co.nz/browse/categorylistings.aspx?condition=all&mcatpath=mobile-phones%2fmobile-phones&page='
           +str(page))
urlmaker(0)


def soupify(page):
    return(BeautifulSoup(opener.open(urlmaker(page)).read(),"lxml") )

In [4]:
import time
start_time = time.time()


soups=[]

for i in range(0,45): #try make this the limit.
        soups.append(soupify(i))

print("--- %s seconds ---" % (time.time() - start_time))        

--- 63.67928981781006 seconds ---


In [5]:
def item(page):
    p=soups[page].findAll('a', {'class':re.compile("tile")})
    paragraphs=[]
    
    for x in p:
        paragraphs.append(str(x))
    for i in range(len(paragraphs)):  
        paragraphs[i]=" ".join(paragraphs[i].split())
    return(paragraphs)

In [6]:
items=[]
for j in range(len(soups)):
        items.append(item(j))
     

In [7]:
def ID(page):
    liste=[]
    for i in range(len(items[page])):
        liste.append(re.search('<div class="watchlist " id="watchlist(.+?)" onclick', 
                               items[page][i]).group(1))  
    return(liste)

In [8]:
IDS=[]

for i in range(len(soups)):
    IDS.append(ID(i))
    
flattened_IDS = [val for sublist in IDS for val in sublist]

In [9]:
Id=np.unique(flattened_IDS).tolist()

In [10]:
def urlmaker2(ID):
    return('http://www.trademe.co.nz/mobile-phones/mobile-phones/auction-'+ID+'.htm')

In [11]:
def soupify2(ID):
    return(BeautifulSoup(opener.open(urlmaker2(ID)).read(),"lxml") )
#soupify2(Id[1])

#Returns .html file for a given page.

In [12]:
import time
start_time = time.time()
soups2=[]

for i in range(len(Id)):
    soups2.append(soupify2(Id[i]))
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 1109.031227350235 seconds ---


In [13]:
for i in range(len(soups2)):
    soups2[i]=str(soups2[i])

In [14]:
for i in range(len(soups2)):  
        soups2[i]=" ".join(soups2[i].split())

In [15]:
PhoneNames=[]
for i in range(len(soups2)):
    PhoneNames.append(re.search('<title>(.+?)Trade Me</title>', 
                               soups2[i]).group(1))
PhoneNames = [x[:-3] for x in PhoneNames]


In [16]:
Region=[]
for i in range(len(soups2)):
    Region.append(re.search('"sellerRegion": "(.+?)", "', 
                               soups2[i]).group(1))

In [17]:
City=[]
for i in range(len(soups2)):
    City.append(re.search('"sellerDistrict": "(.+?)", "', 
                               soups2[i]).group(1))

In [18]:
Brand=[]
for i in range(len(soups2)):
    Brand.append(re.search('"categoryLevel3": "(.+?)",', 
                               soups2[i]).group(1))

In [19]:
Type=[]  #Not all listings have this, so added 'try, except' logic.
for i in range(len(soups2)):
    try:
        Type.append(re.search('"categoryLevel4": "(.+?)",', 
                                   soups2[i]).group(1))
    except:
        Type.append('Other')

In [20]:
SellerFeedBack=[]  #Not all listings have this, so added 'try, except' logic.
for i in range(len(soups2)):
    try:
        SellerFeedBack.append(re.search('<span id="SellerProfile_PercentPositiveFeedback">(.+?)</span>', 
                                   soups2[i]).group(1) +'%')
    except:
        SellerFeedBack.append('None')

In [223]:
SellerName=[]  #Not all listings have this, so added 'try, except' logic.
for i in range(len(soups2)):
    
        if len(re.search('"SellerProfile_MemberNicknameLink">(.+?)</a>', 
                                   soups2[i]).group(1))!=0:
                SellerName.append(re.search('"SellerProfile_MemberNicknameLink">(.+?)</a>', 
                                   soups2[i]).group(1))
        else:
            SellerName.append('holder')


In [102]:
ClosingTime=[]  #Not all listings have this, so added 'try, except' logic.
some_list=['0','1','2','3','4','5','6','7','8','9']
for i in range(len(soups2)):
    stringe=re.search('id="ClosingTime_ClosingTime">(.+?)</span>',
                      soups2[i]).group(1)[1]
    matching = [s for s in some_list if stringe in s]
    if len(matching)==0:
        ClosingTime.append(re.search('id="ClosingTime_TimeLeft">(.+?)</span>',
                      soups2[i]).group(1))
    elif len(matching)!=0:
        ClosingTime.append(re.search('id="ClosingTime_ClosingTime">(.+?)</span>',
                      soups2[i]).group(1))
#ClosingTime

In [104]:
ClosingDate=[]  #Not all listings have this, so added 'try, except' logic.
some_list=['0','1','2','3','4','5','6','7','8','9']
for i in range(len(soups2)):
    stringe=re.search('id="ClosingTime_ClosingTime">(.+?)</span>',
                      soups2[i]).group(1)[1]
    matching = [s for s in some_list if stringe in s]
    if len(matching)!=0:
        ClosingDate.append(re.search('id="ClosingTime_TimeLeft">(.+?)</span>',
                      soups2[i]).group(1))
    elif len(matching)==0:
        ClosingDate.append(re.search('id="ClosingTime_ClosingTime">(.+?)</span>',
                      soups2[i]).group(1))
#ClosingDate

In [144]:
NumberBids=[]  #Not all listings have this, so added 'try, except' logic.
for i in range(len(soups2)):
    try:
        meep=(re.search('id="Bidding_TotalBids">(.+?)<div class="reserve-status', 
                                   soups2[i]).group(1))
        if len(meep)==7:
            NumberBids.append('0')
        elif len(meep)!=7:
            NumberBids.append( re.search('id="Bidding_TotalBids">Total Bids: <strong>(.+?)</strong>',
                                         soups2[i]).group(1))
    except:
        NumberBids.append('Buy Now Only')

In [218]:
BuyNowPrice=[]
for i in range(len(soups2)):
    meep=(re.search('"buyNowPrice": (.+?), "mustPickUp":', 
                                   soups2[i]).group(1))
    if meep=='0.0':
        BuyNowPrice.append('Bid Only')
    elif len(meep)==0:
            BuyNowPrice.append('holder')
    else:
        BuyNowPrice.append('$'+meep)

In [219]:
BidPrice=[]

for i in range(len(soups2)):
    
    try:
        BidPrice.append(re.search('id="Bidding_CurrentBidValue">(.+?)</div>', 
                                   soups2[i]).group(1))
    except:
        BidPrice.append('Buy Now Only')

In [232]:
df=pd.DataFrame()

df['Phone Name']=PhoneNames
df['Brand']=Brand
df['Type']=Type
df['Buynow Price']=BuyNowPrice
df['Bid Price']=BidPrice
df['Number of Bids']=NumberBids
df['Closing Time']=ClosingTime
df['Closing Date']=ClosingDate
df['Region']=Region
df['City']=City
df['Seller Feedback']=SellerFeedBack
df['Seller Name']=SellerName



df.head(15)

,Phone Name,Brand,Type,Buynow Price,Bid Price,Number of Bids,Closing Time,Closing Date,Region,City,Seller Feedback,Seller Name
0,iphone 6 64GB Gold Mint Condition+12 Month Wty,iphone,iphone-6,$599.00,$599.00,0,49 mins,"Thu 29 Jun, 2:10 pm",Auckland,Auckland City,98.9%,future.tech
1,Refurbished Mint Samsung S5 White 4G 16GB 6mon...,samsung,galaxy-s5,$379.00,$379.00,0,49 mins,"Thu 29 Jun, 2:10 pm",Auckland,Auckland City,98.9%,future.tech
2,Iphone 5S 32GB Gold Mint Condition Freeship+6 ...,iphone,iphone-5s,$399.00,$399.00,0,53 mins,"Thu 29 Jun, 2:14 pm",Auckland,Auckland City,98.9%,future.tech
3,IPhone 6S Plus 128GB Gold Refurbished,iphone,iphone-6s-plus,$929.00,$929.00,0,1 hr 9 mins,"Thu 29 Jun, 2:30 pm",Auckland,Auckland City,98.9%,future.tech
4,Samsung Note 3 N9005 32GB Black Mint Condition...,samsung,galaxy-note,$349.00,$349.00,0,1 hr 19 mins,"Thu 29 Jun, 2:40 pm",Auckland,Auckland City,98.9%,future.tech
5,iphone 6 Plus 64GB Gold Mint Condition+12 Mth Wty,iphone,iphone-6-plus,$699.00,$699.00,0,2 hrs 9 mins,"Thu 29 Jun, 3:30 pm",Auckland,Auckland City,98.9%,future.tech
6,iphone 5 64gb iphone 5 64G Black 1 year warranty,iphone,iphone-5,$339.00,$339.00,0,2 hrs 20 mins,"Thu 29 Jun, 3:41 pm",Auckland,Auckland City,99.3%,qllimited
7,iphone 6 Plus 64GB Silver Mint Condition+12 MsWty,iphone,iphone-6-plus,$699.00,$699.00,0,2 hrs 49 mins,"Thu 29 Jun, 4:10 pm",Auckland,Auckland City,98.9%,future.tech
8,iphone 6 128GB Gold Mint Condition+12 Month Wty,iphone,iphone-6,$629.00,$629.00,0,2 hrs 49 mins,"Thu 29 Jun, 4:10 pm",Auckland,Auckland City,98.9%,future.tech
9,Iphone 5s 64GB Black MintCondition Freeship+6 mty,iphone,iphone-5s,$419.00,$419.00,0,2 hrs 49 mins,"Thu 29 Jun, 4:10 pm",Auckland,Auckland City,98.9%,future.tech


In [233]:
import time
timestr = time.strftime("%Y+%m+%d-%H+%M+%S")
timestr

'2017+06+29-16+46+57'

In [234]:
path='D:\\TradeMe2\\'
df.to_csv(path+'SCRAPENEW'+timestr+'.csv', dwdelimiter=',', encoding='utf-8')